In [8]:
import gpxpy
import geopandas as gpd
import pandas as pd
from pygeodesy.ellipsoidalVincenty import LatLon
from pygeodesy import sphericalTrigonometry as st
from geopy.distance import geodesic
from math import atan2, degrees
import folium
import xml.etree.ElementTree as ET
import re
import tkinter as tk
from tkinter import filedialog
import uuid


# Conversion factors
METERS_TO_FEET = 3.28084
KMH_TO_KNOTS = 0.539956803455724
MPS_TO_KMH = 3.6
MPS_TO_KNOTS = 1.944
FLIGHT_SPEED = 25 #Knots - Speed at which assume in flight
POINT_SOURCE = ["AV_Plan","MGL_Odyssey"]
FLIGHT_STATUS = ["BF","IF","AF"]


# Define the namespace for GPX and AvPlan extensions
namespace = {'gpx': 'http://www.topografix.com/GPX/1/1', 'avplan': 'avplan'}


#########################################
# Define Functions BELOW HERE
#########################################

# Function Select File Dialogue function
def select_file(file_type, file_filter):
    # Initialize the Tkinter root window
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    
    # Open the file selection dialog
    file_path = filedialog.askopenfilename(
        title=f"Select a file of type: {file_filter}",
        filetypes=[(file_type, file_filter),("All Files", "*.*")]
    )
    return file_path

# Function Load the GPX file and parse it using gpxpy
def load_gpx_file(gpx_file_path):
    with open(gpx_file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        tree = ET.parse(gpx_file_path)
        root = tree.getroot()
    return gpx, root

# Function extract Leg elements from AVPlan gpx file using the <rte> element
def extract_legs (root, gpx_file_path):
        # Initialize a dictionary to store leg data
    leg_data = {
        'leg_guid': [],
        'leg_name': [],
        'leg_number': [],
        'src': [],
        'desc': [],
        'departure_time': [],
        'holding_time': [],
        'pob': [],
        'taxi_time': [],
        'block_off': [],
        'wheels_on': [],
        'wheels_off': [],
        'taxi_fuel': [],
        'approach_fuel': [],
        'load_ids': [],
        'load_values': [],
        'fuel_load_ids': [],
        'fuel_load_values': [],
        'gpx_file_path': []
    }
    for rte in root.findall('gpx:rte', namespace):
        # Generate a GUID for each leg
        leg_guid = str(uuid.uuid4())

        # Extract the leg name and number
        leg_name = rte.find('gpx:name', namespace).text if rte.find('gpx:name', namespace) is not None else None
        leg_number = rte.find('gpx:number', namespace).text if rte.find('gpx:number', namespace) is not None else None

        # Extract the <AvPlanLegDetails> extension
        leg_details = rte.find('gpx:extensions/avplan:AvPlanLegDetails', namespace)
        if leg_details is not None:
            src = leg_details.find('avplan:src', namespace).text if leg_details.find('avplan:src', namespace) is not None else None
            desc = leg_details.find('avplan:desc', namespace).text if leg_details.find('avplan:desc', namespace) is not None else None
            departure_time = leg_details.find('avplan:AvPlanDepartureTime', namespace).text if leg_details.find('avplan:AvPlanDepartureTime', namespace) is not None else None
            holding_time = leg_details.find('avplan:AvPlanHoldingTime', namespace).text if leg_details.find('avplan:AvPlanHoldingTime', namespace) is not None else None
            pob = leg_details.find('avplan:AvPlanPOB', namespace).text if leg_details.find('avplan:AvPlanPOB', namespace) is not None else None
            taxi_time = leg_details.find('avplan:AvPlanTaxiTime', namespace).text if leg_details.find('avplan:AvPlanTaxiTime', namespace) is not None else None
            block_off = leg_details.find('avplan:AvPlanBlockOff', namespace).text if leg_details.find('avplan:AvPlanBlockOff', namespace) is not None else None
            wheels_on = leg_details.find('avplan:AvPlanWheelsOn', namespace).text if leg_details.find('avplan:AvPlanWheelsOn', namespace) is not None else None
            wheels_off = leg_details.find('avplan:AvPlanWheelsOff', namespace).text if leg_details.find('avplan:AvPlanWheelsOff', namespace) is not None else None
            taxi_fuel = leg_details.find('avplan:AvPlanTaxiFuel', namespace).text if leg_details.find('avplan:AvPlanTaxiFuel', namespace) is not None else None
            approach_fuel = leg_details.find('avplan:AvPlanApproachFuel', namespace).text if leg_details.find('avplan:AvPlanApproachFuel', namespace) is not None else None

            # Extract loading data
            load_ids = []
            load_values = []
            for load in leg_details.findall('avplan:AvPlanLoading/avplan:AvPlanLoad', namespace):
                load_id = load.find('avplan:AvPlanLoadID', namespace).text if load.find('avplan:AvPlanLoadID', namespace) is not None else None
                load_value = load.find('avplan:AvPlanLoadValue', namespace).text if load.find('avplan:AvPlanLoadValue', namespace) is not None else None
                load_ids.append(load_id)
                load_values.append(load_value)

            # Extract fuel loading data
            fuel_load_ids = []
            fuel_load_values = []
            for fuel_load in leg_details.findall('avplan:AvPlanFuelLoading/avplan:AvPlanFuelLoad', namespace):
                fuel_load_id = fuel_load.find('avplan:AvPlanFuelLoadID', namespace).text if fuel_load.find('avplan:AvPlanFuelLoadID', namespace) is not None else None
                fuel_load_value = fuel_load.find('avplan:AvPlanFuelLoadValue', namespace).text if fuel_load.find('avplan:AvPlanFuelLoadValue', namespace) is not None else None
                fuel_load_ids.append(fuel_load_id)
                fuel_load_values.append(fuel_load_value)

            # Append the extracted data to the dictionary
            leg_data['leg_guid'].append(leg_guid)
            leg_data['leg_name'].append(leg_name)
            leg_data['leg_number'].append(leg_number)
            leg_data['src'].append(src)
            leg_data['desc'].append(desc)
            leg_data['departure_time'].append(departure_time)
            leg_data['holding_time'].append(holding_time)
            leg_data['pob'].append(pob)
            leg_data['taxi_time'].append(taxi_time)
            leg_data['block_off'].append(block_off)
            leg_data['wheels_on'].append(wheels_on)
            leg_data['wheels_off'].append(wheels_off)
            leg_data['taxi_fuel'].append(taxi_fuel)
            leg_data['approach_fuel'].append(approach_fuel)
            leg_data['load_ids'].append(load_ids)
            leg_data['load_values'].append(load_values)
            leg_data['fuel_load_ids'].append(fuel_load_ids)
            leg_data['fuel_load_values'].append(fuel_load_values)
            leg_data['gpx_file_path'].append(gpx_file_path)

    # Convert the dictionary into a pandas DataFrame
    leg_df = pd.DataFrame(leg_data)

    return leg_df

# Function Extract route points from an AVPlan gpx file
def extract_routes(root, leg_guid):
    # Initialize a dictionary to store route point data
    route_data = {
        'leg_guid': [],
        'latitude': [],
        'longitude': [],
        'name': [],
        'type': [],
        'altitude': [],
        'description': [],
        'magvar': [],
        'delay': [],
        'alternate': [],
        'lsalt': [],
        'segment_rules': [],
        'track': [],
        'distance': [],
        'distance_remaining': [],
        'eta': [],
        'ata': [],
        'actual_fob': []
    }

    # Loop through all the <rtept> elements in the GPX file
    for rtept in root.findall('gpx:rte/gpx:rtept', namespace):
        lat = rtept.get('lat')
        lon = rtept.get('lon')
        name = rtept.find('gpx:name', namespace).text if rtept.find('gpx:name', namespace) is not None else None
        wpt_type = rtept.find('gpx:type', namespace).text if rtept.find('gpx:type', namespace) is not None else None

        # Find the <AvPlanWaypointDetails> extension within <extensions>
        waypoint_details = rtept.find('gpx:extensions/avplan:AvPlanWaypointDetails', namespace)
        if waypoint_details is not None:
            altitude = waypoint_details.find('avplan:AvPlanAltitude', namespace).text if waypoint_details.find('avplan:AvPlanAltitude', namespace) is not None else None
            description = waypoint_details.find('avplan:desc', namespace).text if waypoint_details.find('avplan:desc', namespace) is not None else None
            magvar = waypoint_details.find('avplan:magvar', namespace).text if waypoint_details.find('avplan:magvar', namespace) is not None else None
            delay = waypoint_details.find('avplan:AvPlanDelay', namespace).text if waypoint_details.find('avplan:AvPlanDelay', namespace) is not None else None
            alternate = waypoint_details.find('avplan:AvPlanAlternate', namespace).text if waypoint_details.find('avplan:AvPlanAlternate', namespace) is not None else None
            lsalt = waypoint_details.find('avplan:AvPlanLSALT', namespace).text if waypoint_details.find('avplan:AvPlanLSALT', namespace) is not None else None
            segment_rules = waypoint_details.find('avplan:AvPlanSegRules', namespace).text if waypoint_details.find('avplan:AvPlanSegRules', namespace) is not None else None
            track = waypoint_details.find('avplan:AvPlanTrack', namespace).text if waypoint_details.find('avplan:AvPlanTrack', namespace) is not None else None
            distance = waypoint_details.find('avplan:AvPlanDistance', namespace).text if waypoint_details.find('avplan:AvPlanDistance', namespace) is not None else None
            distance_remaining = waypoint_details.find('avplan:AvPlanDistanceRemain', namespace).text if waypoint_details.find('avplan:AvPlanDistanceRemain', namespace) is not None else None
            eta = waypoint_details.find('avplan:AvPlanETA', namespace).text if waypoint_details.find('avplan:AvPlanETA', namespace) is not None else None
            ata = waypoint_details.find('avplan:AvPlanATA', namespace).text if waypoint_details.find('avplan:AvPlanATA', namespace) is not None else None
            actual_fob = waypoint_details.find('avplan:AvPlanActualFOB', namespace).text if waypoint_details.find('avplan:AvPlanActualFOB', namespace) is not None else None

            # Append the extracted data to the dictionary
            route_data['leg_guid'].append(leg_guid)
            route_data['latitude'].append(float(lat))
            route_data['longitude'].append(float(lon))
            route_data['name'].append(name)
            route_data['type'].append(wpt_type)
            route_data['altitude'].append(float(altitude) if altitude else None)
            route_data['description'].append(description)
            route_data['magvar'].append(float(magvar) if magvar else None)
            route_data['delay'].append(float(delay) if delay else None)
            route_data['alternate'].append(float(alternate) if alternate else None)
            route_data['lsalt'].append(float(lsalt) if lsalt else None)
            route_data['segment_rules'].append(segment_rules)
            route_data['track'].append(float(track) if track else None)
            route_data['distance'].append(float(distance) if distance else None)
            route_data['distance_remaining'].append(float(distance_remaining) if distance_remaining else None)
            route_data['eta'].append(eta)
            route_data['ata'].append(ata)
            route_data['actual_fob'].append(float(actual_fob) if actual_fob else None)

    # Convert the dictionary into a pandas DataFrame
    route_df = pd.DataFrame(route_data)

    return route_df

#Function extract track points from GPX file
def extract_track (gpx, leg_guid):
    # Initialize lists to store extracted data
    data = {
        'leg_guid': [],
        'latitude': [],
        'longitude': [],
        'elevation_ft': [],
        'time': [],
        'speed_avplan': [],
        'heading': [],
        'hdop': [],
        'vdop': [],
        'geometry': []
    }

    # Extract the track points from the GPX file
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data['leg_guid'].append(leg_guid)
                data['latitude'].append(point.latitude)
                data['longitude'].append(point.longitude)
                data['elevation_ft'].append(point.elevation)  # Assume elevation is already in feet
                data['time'].append(point.time)

                # Extract HDOP, VDOP, and heading from the extensions if available
                hdop = None
                vdop = None
                heading = None
                speed_knots = None

                if point.extensions:
                    for ext in point.extensions:
                        tree = ET.ElementTree(ET.fromstring(ET.tostring(ext)))
                        root = tree.getroot()

                        # Extract speed (knots), heading, hdop, and vdop from extensions
                        speed_elem = root.find('.//{avplan}speed')
                        heading_elem = root.find('.//{avplan}heading')
                        hdop_elem = root.find('.//hdop')
                        vdop_elem = root.find('.//vdop')

                        if speed_elem is not None:
                            speed_avplan = float(speed_elem.text)
                        if heading_elem is not None:
                            heading = float(heading_elem.text)
                        if hdop_elem is not None:
                            hdop = float(hdop_elem.text)
                        if vdop_elem is not None:
                            vdop = float(vdop_elem.text)

                data['speed_avplan'].append(speed_avplan)
                data['heading'].append(heading)
                data['hdop'].append(hdop)
                data['vdop'].append(vdop)
                data['geometry'].append((point.longitude, point.latitude))  # Store geometry as (lon, lat)

    # Convert to a pandas DataFrame
    df = pd.DataFrame(data)
    return df

# Function Compute distance, time difference, elevation change, and angle of elevation between consecutive points
def compute_track_detail(gdf):
    # Initialize lists for distance, time difference, elevation change, and angle of elevation
    distances = [0]
    speed_calc = [0]
    time_diffs = [0]
    elevation_changes = [0]
    angles = [0]
    point_source = [POINT_SOURCE[0]]
    in_flight = [None]
    flight_status = FLIGHT_STATUS[0]
    #Cycle through all points after the first one
    for i in range(1, len(gdf)):
        # Get the current and previous points
        point1 = (gdf.iloc[i-1]['latitude'], gdf.iloc[i-1]['longitude'])
        point2 = (gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude'])

        # Calculate distance using geodesic method (in meters)
        dist = geodesic(point1, point2).meters
        distances.append(dist)

        # Calculate time difference in seconds
        time_diff = (gdf.iloc[i]['time'] - gdf.iloc[i-1]['time']).total_seconds()
        time_diffs.append(time_diff)

        # Calculate speed (m/s) as distance divided by time, and convert to km/h and knots
        if time_diff > 0:
            speed_mps = (dist / time_diff)
        else:
            speed_mps = 0
        speed_calc.append(speed_mps)
        if gdf.iloc[i]['speed_avplan'] >= FLIGHT_SPEED and flight_status != FLIGHT_STATUS[2]:
            in_flight.append(True)
            flight_status = FLIGHT_STATUS[1]
        else:
            in_flight.append(None)
            if flight_status == FLIGHT_STATUS[1]:
                flight_status = FLIGHT_STATUS[2]
        # Calculate elevation change (in feet)
        elevation_change = gdf.iloc[i]['elevation_ft'] - gdf.iloc[i-1]['elevation_ft']
        elevation_changes.append(elevation_change)
        # Calculate angle of elevation (in degrees)
        if dist > 0:  # Avoid division by zero
            angle = degrees(atan2(elevation_change, dist))  # Use atan2 for angle
        else:
            angle = 0
        angles.append(angle)       
        point_source.append(POINT_SOURCE[0])

    # Add the calculated values to the GeoDataFrame
    gdf['distance_m'] = distances
    gdf['speed_calc'] = speed_calc
    gdf['time_diff_sec'] = time_diffs
    gdf['elevation_change_ft'] = elevation_changes
    gdf['angle_deg'] = angles
    gdf['in_flight'] = in_flight
    gdf['point_source'] = point_source
    return gdf

# Function to find the two closest points on the track
def find_closest_points(gate_point, gdf):
    gate_lat, gate_lon = gate_point['latitude'], gate_point['longitude']
    
    # Calculate distances from the gate_point to all points in the GeoDataFrame
    distances = gdf.apply(lambda row: geodesic((gate_lat, gate_lon), (row['latitude'], row['longitude'])).meters, axis=1)

    # Find the index of the closest point
    closest_index = distances.idxmin()


    # Find the second & third closest point (before and after the closest)
    if closest_index == 0:
        closest_index = 1
        second_closest_index = 0
        third_closest_index = 2
    elif closest_index == len(gdf) - 1:
        closest_index = len(gdf) - 2
        second_closest_index = len(gdf) - 3
        third_closest_index = -1
    else:
        second_closest_index = closest_index - 1
        third_closest_index = closest_index + 1
    
    return closest_index, second_closest_index, third_closest_index


#Function Interpolate the closest point on the geodesic line (great circle) between two track points to the gate point.
def interpolate_between_points(gate_point, gdf, idx1, idx2, idx3):
    #Interpolate the closest point on the geodesic line (great circle) between two track points to the gate point.
    #Uses PyGeodesy to calculate the closest point along the geodesic line.

    # Extract the two closest points from the GeoDataFrame
    point1 = gdf.iloc[idx1]
    point2 = gdf.iloc[idx2]
    point3 = gdf.iloc[idx3]
    
    # Use PyGeodesy to find the nearest point on the geodesic between point1 and point2
    #nearest = gate_geo.nearestOn3(point1_geo, point2_geo)
    
    # Create PyGeodesy LatLon objects for the track points and gate point
    point1_geo = st.LatLon(point1['latitude'], point1['longitude'])
    point2_geo = st.LatLon(point2['latitude'], point2['longitude'])
    point3_geo = st.LatLon(point3['latitude'], point3['longitude'])
    gate_geo = st.LatLon(gate_point['latitude'], gate_point['longitude'])

    # Use PyGeodesy to find the nearest point on the geodesic between point1 and point2
    #nearest = gate_geo.nearestOn(point1_geo, point2_geo, point3_geo)
    #print(nearest)
    nearest = [st.nearestOn3(gate_geo, [point1_geo,point2_geo,point3_geo])] 
    nearest_geo = nearest[0].closest
    
    # Interpolate time between the two points
    time_diff = (point3['time'] - point2['time']).total_seconds()
    dist_diff = point3_geo.distanceTo(point2_geo)
    dist_to = point2_geo.distanceTo(nearest_geo)
    fraction_of_distance = dist_to/dist_diff
    interpolated_time = point2['time'] + pd.Timedelta(seconds=time_diff * fraction_of_distance)

    # Compute distance from the gate point to the nearest point on the geodesic
    distance_from_gate = gate_geo.distanceTo(nearest_geo)

    # Return the results in a dictionary
    return {
        'gate_lat': gate_point['latitude'],
        'gate_lon': gate_point['longitude'],
        'interpolated_lat': nearest_geo.lat,
        'interpolated_lon': nearest_geo.lon,
        'interpolated_time': interpolated_time,
        'distance_from_gate_m': distance_from_gate
    }

# Function to check if a name starts with "in" or "out" followed by a number
def is_in_or_out_with_number(name):
    # Use regex to match "in" or "out" followed by a number
    return bool(re.match(r'^(in|out)\d+', name.lower()))

# Function to process each gate point and interpolate
def compute_track_points_for_gate(gate_points, gdf):
    interpolated_results = []

    for gate_point in gate_points:
        idx1, idx2, idx3 = find_closest_points(gate_point, gdf)
        result = interpolate_between_points(gate_point, gdf, idx1, idx2, idx3)
        interpolated_results.append(result)

    return pd.DataFrame(interpolated_results)

# Function Extract the gate points from the route data. Skip the first and last route points and use all the others as gates
def extract_gate_points(route_df):
    gate_points = []
    for index, row in route_df.iloc[1:-1].iterrows():
        # Create a dictionary with the latitude, longitude, and elevation
        gate_point = {
            'latitude': row['latitude'],
            'longitude': row['longitude'],
            'elevation': row['altitude']  # Assuming 'altitude' is the correct field for elevation
        }
        # Add the gate point to the list
        gate_points.append(gate_point)
    return gate_points

#Function display Leg Map with all gate details
def display_gate_details_map (route_df, gdf, result):
    # Previous point's coordinates, initialized to None
    previous_point = None
    # Create a Folium map centered on the first point
    m = folium.Map(location=[gdf.iloc[0]['latitude'], gdf.iloc[0]['longitude']], zoom_start=10)

    # Plot each route point
    for index, row in route_df.iterrows():
        # Determine the icon color: red for points with names starting with "in" or "out" followed by a number, otherwise orange
        icon_color = 'green' if is_in_or_out_with_number(row['name']) else 'orange'

        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=(f"Name: {row['name']}<br>Type: {row['type']}<br>Altitude: {row['altitude']} ft<br>Description: {row['description']}"),
            icon=folium.Icon(color=icon_color, icon='info-sign')
        ).add_to(m)
            # Draw a line between the previous point and the current point if previous point exists
        if previous_point is not None:
            # Create a popup for the line with heading and distance
            # Draw a line (Polyline) between the previous point and the current point
            folium.PolyLine(
                locations=[previous_point, [row['latitude'], row['longitude']]],
                color='magenta',  # Line color
                weight=3,  # Thickness of the line
            ).add_to(m)
    
        # Update the previous_point to the current point for the next iteration
        previous_point = [row['latitude'], row['longitude']]

    # Add the GPS track to the map
    for i, row in gdf.iterrows():
        track_color = 'blue' if row['in_flight'] else 'yellow'
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=2,  # Adjust the size of the circle
            color= track_color,  # Outline color of the circle
            fill=True,
            fill_color= track_color,  # Fill color of the circle
            fill_opacity=0.5,  # Adjust the opacity of the fill
            popup=(
                f"Id: {i} <br>"
                f"Elevation: {row['elevation_ft']} ft<br>"
                f"Speed Av: {row['speed_avplan'] * MPS_TO_KNOTS:.2f} knots<br>"
                f"Speed Cal: {row['speed_calc'] * MPS_TO_KNOTS:.2f} knots<br>"
                f"Heading: {row['heading']}°<br>"
                f"Distance: {row['distance_m']} m<br>"
                f"Interval: {row['time_diff_sec']}<br>"
                f"Elevation change: {row['elevation_change_ft']} ft<br>"
                f"Angle: {row['angle_deg']}°<br>"
                f"Flight: {row['in_flight']}°<br>"
                f"Time: {row['time']}")
        ).add_to(m)
    
    # Add interpolated point to the map (red marker)
    for index, row in result.iterrows():
        folium.CircleMarker(
            location=[row['interpolated_lat'], row['interpolated_lon']],
            radius=8,  # Adjust the size of the circle
            color='red',  # Outline color of the circle
            fill=True,
            fill_color='red',  # Fill color of the circle
            fill_opacity=0.5,  # Adjust the opacity of the fill# Accessing values from the current row
            popup=(
                f"Interpolated Point<br>Coordinates: ({row['interpolated_lat']}, {row['interpolated_lon']})<br>"
                f"Time: {row['interpolated_time']}<br>"
                f"Distance from gate: {row['distance_from_gate_m']} meters"
            )
        ).add_to(m)

    return m

#########################################
# Main Code BELOW HERE
#########################################

# Call the file selection dialog
gpx_file_path = select_file("GPX File", "*.gpx")
# Print the selected file path
if gpx_file_path:
    print(f"Selected file: {gpx_file_path}")
else:
    print("No file selected")
        
# Load the GPX data from selected file
gpx, gpx_root = load_gpx_file(gpx_file_path)
leg_av_df = extract_legs (gpx_root, gpx_file_path)
#print(leg_av_df)

route_av_df = extract_routes(gpx_root,leg_av_df.leg_guid)
#print(route_av_df)

track_av_df = extract_track(gpx,leg_av_df.leg_guid)
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    track_av_df, geometry=gpd.points_from_xy(track_av_df['longitude'], track_av_df['latitude'])
)

# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)
gdf = compute_track_detail(gdf)

#print(gdf)

gate_points = extract_gate_points(route_av_df)  
    
# Call the function to compute track points and plot them on the map
interpolated_gate_points = compute_track_points_for_gate(gate_points, gdf)

#Display Map
map = display_gate_details_map (route_av_df, gdf[gdf['in_flight'].notnull()], interpolated_gate_points)
# Save the map to an HTML file
map.save('map_with_gates_and_interpolated_points.html')
map

print(route_av_df)



No file selected


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
gpx, gpx_root = load_gpx_file(gpx_file_path)
leg_av_df = extract_legs (gpx_root,gpx_file_path)
print(leg_av_df)
route_av_df = extract_routes(gpx_root,leg_av_df.leg_guid)
print(route_av_df)

track_av_df = extract_track(gpx,leg_av_df.leg_guid)
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    track_av_df, geometry=gpd.points_from_xy(track_av_df['longitude'], track_av_df['latitude'])
)

# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)
gdf = compute_track_detail(gdf)
print(gdf)

In [ ]:
print(route_av_df)